In [6]:
from pandas import MultiIndex, Int16Dtype
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
sns.set_style('darkgrid')

import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import joblib

from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, Binarizer, RobustScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, PowerTransformer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.compose import ColumnTransformer


from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import ElasticNet, SGDRegressor, LinearRegression, Lasso
from sklearn.svm import SVR
from sklearn.linear_model import BayesianRidge,LinearRegression, LogisticRegression
from sklearn.kernel_ridge import KernelRidge
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.model_selection import KFold, ShuffleSplit, LeaveOneOut, StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import joblib

# Warnings configuration
# ==============================================================================
import warnings
# warnings.filterwarnings('ignore')

%run "../1. Librerias Mongo/MongoDB_Connections.ipynb"
%run "../1. Librerias Mongo/MongoDB_Funciones_Consultas.ipynb"

In [7]:
datos = pd.read_csv("../../Data/DataFrame_Final_Cierre_Cluster.csv",parse_dates=['FECHA'])

### Función de nueva fecha

In [9]:
# Meteorologia

db_Meteo = _connect_mongo('cloud', 'cluster0.15npsxw.mongodb.net', None, 'ucmtfm2022', 'UCM_2022', 'BiciMAD', 'Meteo_US_NivelHora')

data_Meteo = _consulta_meteoUS_full(db_Meteo)

# Se agrega campo FECHA para join
data_Meteo['FECHA'] =pd.to_datetime({'year': data_Meteo['ANIO'],
                                          'month': data_Meteo["MES"],
                                          'day':  data_Meteo['DIA']},
                                          format='%d-%m-%Y', errors='coerce')

data_Meteo['TEMPERATURA'] = pd.to_numeric(data_Meteo['TEMPERATURA'])
data_Meteo['VIENTO'] = pd.to_numeric(data_Meteo['VIENTO'])
data_Meteo['PRESION'] = pd.to_numeric(data_Meteo['PRESION'])
data_Meteo['HUMEDAD'] = pd.to_numeric(data_Meteo['HUMEDAD'])
data_Meteo['PRECIPITACION_1h'] = pd.to_numeric(data_Meteo['PRECIPITACION_1h'])
data_Meteo['PRECIPITACION_3h'] = pd.to_numeric(data_Meteo['PRECIPITACION_3h'])

data_Meteo = data_Meteo.groupby(['ANIO', 'MES', 'DIA', 'FECHA']).agg(TEMP_MAX= ('TEMPERATURA','max'),
                                 TEMP_MIN = ('TEMPERATURA','min'),
                                 HUMEDAD = ('HUMEDAD','mean'),
                                 VIENTO = ('VIENTO','mean'),
                                 PRESION = ('PRESION', 'mean'),
                                 PRECIPITACION_1h = ('PRECIPITACION_1h', 'mean'),
                                 PRECIPITACION_3h = ('PRECIPITACION_3h', 'mean'),
                                 DESC_TIEMPO = ('DESC_TIEMPO', lambda x: x.value_counts().index[0]))

data_Meteo.reset_index(inplace=True)

data_Meteo.info()

Conexion OK
Collection(Database(MongoClient(host=['ac-x1d17w2-shard-00-00.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-02.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-01.15npsxw.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-xyv6ql-shard-0', tls=True, serverselectiontimeoutms=4000, tlscafile='C:\\ProgramData\\Anaconda3\\lib\\site-packages\\certifi\\cacert.pem'), 'BiciMAD'), 'Meteo_US_NivelHora')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 0 to 1095
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ANIO              1096 non-null   int64         
 1   MES               1096 non-null   int64         
 2   DIA               1096 non-null   int64         
 3   FECHA             1096 non-null   datetime64[ns]
 4   TEMP_MAX          1096 non-null   float64       
 5   TEMP_MIN          1096 non-null   float6

In [10]:
data_Meteo

,ANIO,MES,DIA,FECHA,TEMP_MAX,TEMP_MIN,HUMEDAD,VIENTO,PRESION,PRECIPITACION_1h,PRECIPITACION_3h,DESC_TIEMPO
0,2018,1,1,2018-01-01,10.98,2.66,68.458333,6.308333,1031.041667,0.000000,0.000000,Clear
1,2018,1,2,2018-01-02,12.69,7.24,71.833333,5.454583,1031.708333,0.000000,0.000000,Clouds
2,2018,1,3,2018-01-03,13.58,7.58,91.541667,4.460000,1031.583333,0.000000,0.000000,Clouds
3,2018,1,4,2018-01-04,13.99,6.58,86.625000,4.929583,1026.333333,0.000000,0.000000,Clouds
4,2018,1,5,2018-01-05,10.88,6.47,84.958333,5.355417,1011.958333,0.242500,0.068750,Clouds
...,...,...,...,...,...,...,...,...,...,...,...,...
1091,2020,12,27,2020-12-27,6.83,-2.12,74.083333,5.469167,1012.916667,0.000000,0.000000,Clear
1092,2020,12,28,2020-12-28,9.13,3.94,64.541667,9.405000,1002.208333,0.123750,0.114583,Clouds
1093,2020,12,29,2020-12-29,7.67,1.08,73.250000,6.263750,1005.375000,0.000000,0.000000,Clouds
1094,2020,12,30,2020-12-30,8.18,1.14,63.166667,5.185833,1015.083333,0.000000,0.000000,Clear


In [59]:
# dataDummy.TEMPORADA.unique()
# dataDummy.DIA_SEMANA.unique()
# dataDummy.Es_Festivo.unique()
# dataDummy.Es_FinSemana.unique()
# dataDummy.DESC_TIEMPO.unique()

In [61]:
# Generación de datos para Dummies

dataDummy = _agregaRegistrosParaDummies(datos.TEMPORADA.unique(), datos.DESC_TIEMPO.unique())

cat_cols= ['MES', 'DIA_SEMANA', 'TEMPORADA', 'Es_Festivo', 'Es_FinSemana','DESC_TIEMPO']
num_cols= ['TEMP_MAX','TEMP_MIN','HUMEDAD','VIENTO','PRESION']

In [133]:
tmp_fec=pd.to_datetime('01/08/2020', format='%d/%m/%Y')
tmp_fec_str=tmp_fec.strftime('%d/%m/%Y')

print(tmp_fec)
print(tmp_fec_str)

data_Meteo[data_Meteo['FECHA']==str(tmp_fec_str)]

# print(df_festivos[df_festivos['FECHA']==str(tmp_fec)])
# df_festivos

print(tmp_fec.dayofweek)
tmp_fec.day_name()





2020-08-01 00:00:00
01/08/2020
5


'Saturday'

In [159]:
fechaPred = _estacionDataFechaNueva(145, pd.to_datetime('01/08/2020', format='%d/%m/%Y'), 
                                    data_Meteo, df_festivos, dataDummy, cat_cols, num_cols)

fechaPred.info()
# fechaPred = pd.DataFrame(fechaPred)
# fechaPred.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   TEMP_MAX                  1 non-null      float64
 1   TEMP_MIN                  1 non-null      float64
 2   HUMEDAD                   1 non-null      float64
 3   VIENTO                    1 non-null      float64
 4   PRESION                   1 non-null      float64
 5   PRECIPITACION_1h          1 non-null      float64
 6   PRECIPITACION_3h          1 non-null      float64
 7   MES_2                     1 non-null      float64
 8   MES_3                     1 non-null      float64
 9   MES_4                     1 non-null      float64
 10  MES_5                     1 non-null      float64
 11  MES_6                     1 non-null      float64
 12  MES_7                     1 non-null      float64
 13  MES_8                     1 non-null      float64
 14  MES_9         

C:\Users\gonza\AppData\Roaming\Python\Python39\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


In [160]:
fechaPred

,TEMP_MAX,TEMP_MIN,HUMEDAD,VIENTO,PRESION,PRECIPITACION_1h,PRECIPITACION_3h,MES_2,MES_3,MES_4,...,TEMPORADA_VERANO,Es_Festivo_1,Es_FinSemana_1,DESC_TIEMPO_Clouds,DESC_TIEMPO_Drizzle,DESC_TIEMPO_Fog,DESC_TIEMPO_Mist,DESC_TIEMPO_Rain,DESC_TIEMPO_Snow,DESC_TIEMPO_Thunderstorm
0,109.981817,109.981817,109.981817,109.981817,109.981817,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [136]:
model = joblib.load('../Modelos/Modelo_145.pkl')

In [161]:
prediccion = model.predict(fechaPred)

In [162]:
prediccion

array([38.04971763])